In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchp

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import round, col
spark = SparkSession.builder.appName("Vine_Review").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [4]:
# Read in the Review dataset as a DataFrame
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
df.write.csv('/content/sample.csv')

In [6]:
# Step 1: Filter total votes count that is equal to or greater than 20
filtered_vote_df = df.filter((df.total_votes>=20))
filtered_vote_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36886257|R1T4FVSABO0IDP|B00V03D9KY|     966609438|Lukas Lk-7950 WD ...|      Automotive|          5|            8|         31|   N|                Y|      Great Value!!!|LK-7950 is an out...| 2015-08-31|
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|    

In [7]:
# Step 2: Filter dataframe created in Step1 to retrieve all rows where the numbers of helpful_votes divided by total_votes =>50%
filtered_vote_percentage_df = filtered_vote_df.filter(((filtered_vote_df.helpful_votes)/(filtered_vote_df.total_votes))>=0.5)
filtered_vote_percentage_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [8]:
# Step 3: Filter df in step 2 to retrieve reviews as part of the Vine program (vine =='Y')
filtered_vine_y_df = filtered_vote_percentage_df.filter((filtered_vote_percentage_df.vine == 'Y'))
#.drop_duplicates(['product_id'])
filtered_vine_y_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35560012| REKN4PLM5174P|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|          219|        225|   Y|                N|Head 2 Head, It's...|Having come into ...| 2015-07-08|
|         US|   52693410| RQOG8IIJX7CKB|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|    

In [ ]:
# Step 4: Repeat step 3 but filter by non-vine program (vine == 'N')
filtered_vine_n_df = filtered_vote_percentage_df.filter((filtered_vote_percentage_df.vine == 'N'))
filtered_vine_n_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [ ]:
# Step 5: Determine total number of reviews, 5-star reviews, and % of 5-star reviews for the two types of review (paid vs unpaid)
total_vine_y = filtered_vine_y_df.groupby("product_category").count().withColumnRenamed('count','Total Vine Count')
total_vine_y.show()

+----------------+----------------+
|product_category|Total Vine Count|
+----------------+----------------+
|      Automotive|              82|
+----------------+----------------+



In [ ]:
total_vine_y_star = filtered_vine_y_df.groupby("star_rating").count().sort('star_rating', ascending=False).withColumnRenamed('count','Total_Vine_Customers')
total_vine_y_star.show()



+-----------+--------------------+
|star_rating|Total_Vine_Customers|
+-----------+--------------------+
|          5|                  33|
|          4|                  20|
|          3|                  18|
|          2|                   8|
|          1|                   3|
+-----------+--------------------+



In [ ]:
vine_y_fivestar_df = total_vine_y_star.selectExpr('*', 'Total_Vine_Customers/82*100 as col3').withColumnRenamed('col3','Vine Star Review (%)')
vine_y_fivestar_df.show()

+-----------+--------------------+--------------------+
|star_rating|Total_Vine_Customers|Vine Star Review (%)|
+-----------+--------------------+--------------------+
|          5|                  33|  40.243902439024396|
|          4|                  20|  24.390243902439025|
|          3|                  18|  21.951219512195124|
|          2|                   8|    9.75609756097561|
|          1|                   3|  3.6585365853658534|
+-----------+--------------------+--------------------+



In [ ]:
vine_y_fivestar_df.select("*", round(col('Vine Star Review (%)'),2)).show()

+-----------+--------------------+--------------------+------------------------------+
|star_rating|Total_Vine_Customers|Vine Star Review (%)|round(Vine Star Review (%), 2)|
+-----------+--------------------+--------------------+------------------------------+
|          5|                  33|  40.243902439024396|                         40.24|
|          4|                  20|  24.390243902439025|                         24.39|
|          3|                  18|  21.951219512195124|                         21.95|
|          2|                   8|    9.75609756097561|                          9.76|
|          1|                   3|  3.6585365853658534|                          3.66|
+-----------+--------------------+--------------------+------------------------------+



In [ ]:
total_vine_n = filtered_vine_n_df.groupby("product_category").count().withColumnRenamed('count','Total Non-Vine Count')
total_vine_n.show()

+----------------+--------------------+
|product_category|Total Non-Vine Count|
+----------------+--------------------+
|      Automotive|               24742|
+----------------+--------------------+



In [ ]:
total_vine_n_star = filtered_vine_n_df.groupby("star_rating").count().sort('star_rating', ascending=False).withColumnRenamed('count','Total_Non_Vine_Customers')
total_vine_n_star.show()


+-----------+------------------------+
|star_rating|Total_Non_Vine_Customers|
+-----------+------------------------+
|          5|                   12807|
|          4|                    3634|
|          3|                    1997|
|          2|                    1475|
|          1|                    4829|
+-----------+------------------------+



In [ ]:
vine_n_fivestar_df = total_vine_n_star.selectExpr('*', 'Total_Non_Vine_Customers/24742*100 as col3').withColumnRenamed('col3','Non-Vine Star Review (%)')
vine_n_fivestar_df.show()

+-----------+------------------------+------------------------+
|star_rating|Total_Non_Vine_Customers|Non-Vine Star Review (%)|
+-----------+------------------------+------------------------+
|          5|                   12807|       51.76218575701237|
|          4|                    3634|      14.687575782070972|
|          3|                    1997|       8.071295772370867|
|          2|                    1475|       5.961522916498263|
|          1|                    4829|      19.517419772047532|
+-----------+------------------------+------------------------+



In [ ]:
vine_n_fivestar_df.select("*", round(col('Non-Vine Star Review (%)'),2)).show()

+-----------+------------------------+------------------------+----------------------------------+
|star_rating|Total_Non_Vine_Customers|Non-Vine Star Review (%)|round(Non-Vine Star Review (%), 2)|
+-----------+------------------------+------------------------+----------------------------------+
|          5|                   12807|       51.76218575701237|                             51.76|
|          4|                    3634|      14.687575782070972|                             14.69|
|          3|                    1997|       8.071295772370867|                              8.07|
|          2|                    1475|       5.961522916498263|                              5.96|
|          1|                    4829|      19.517419772047532|                             19.52|
+-----------+------------------------+------------------------+----------------------------------+

